In [46]:
import ast
import tokenize
from io import BytesIO
from typing import List, Dict, Union, Optional, Any, Type, Callable
from typing_extensions import TypedDict

from file import get_file_paths

PROJET_DIR = "./MetaGPT/metagpt"
SUPPORTED_FILE_TYPES = [".py"]

class DependencyData(TypedDict):
    modules: List[str]
    functions_called: List[str]
    global_vars: List[str]
    classes_used: List[str]
    
class BaseExtractor:
    @classmethod
    def extract(cls, node: ast.AST) -> List[str]:
        raise NotImplementedError
    
class ImportExtractor(BaseExtractor):
    @classmethod
    def extract(cls, node: ast.AST) -> List[str]:
        if isinstance(node, ast.Import):
            return [n.name for n in node.names]
        elif isinstance(node, ast.ImportFrom):
            return [f"{node.module}.{n.name}" for n in node.names]
        return []
    
class FunctionCallExtractor(BaseExtractor):
    @classmethod
    def extract(cls, node: ast.AST) -> List[str]:
        if isinstance(node, ast.Call):
            if isinstance(node.func, ast.Name):
                return [node.func.id]
            elif isinstance(node.func, ast.Attribute):
                return [node.func.attr]
        return []
    
class GlobalVarExtractor(BaseExtractor):
    @classmethod
    def extract(cls, node: ast.AST) -> List[str]:
        return node.names if isinstance(node, ast.Global) else []

class ClassUsageExtractor(BaseExtractor):
    @classmethod
    def extract(cls, node: ast.AST) -> List[str]:
        return [node.name] if isinstance(node, ast.ClassDef) else []
    

class CodeAnalyzer:
    def __init__(self, code: str):
        self.code = code
        self.tree = ast.parse(self.code)
    
    def _node_filter(self, node_type: Type) -> Callable[[ast.AST], bool]:
        """Higher-order function that returns a function to filter AST nodes."""
        return lambda node: isinstance(node, node_type)

    def _extract_code_by_tokenizing(self, name: str, node_type: Type) -> Optional[str]:
        """Retrieve the code of a function or class using tokenization."""
        tokens = list(tokenize.tokenize(BytesIO(self.code.encode('utf-8')).readline))
        start_token_index = None
        end_token_index = None

        for index, token in enumerate(tokens):
            # Check if we are at the start of the function or class
            if (token.type == tokenize.NAME and token.string in (node_type.__name__, 'def', 'async def') 
                and tokens[index + 1].type == tokenize.NAME and tokens[index + 1].string == name):
                start_token_index = token.start[0] - 1
            
            # Find the end of the function or class (by matching indentation)
            if start_token_index is not None and token.type == tokenize.DEDENT:
                end_token_index = token.end[0] - 1
                break

        # Return the code segment
        if start_token_index is not None and end_token_index is not None:
            return '\n'.join(self.code.splitlines()[start_token_index:end_token_index])
        
        return None

    def get_function_code(self, name: str) -> Optional[str]:
        return self._extract_code_by_tokenizing(name, ast.FunctionDef)

    def get_class_code(self, name: str) -> Optional[str]:
        return self._extract_code_by_tokenizing(name, ast.ClassDef)
    

    def analyze(self) -> DependencyData:
        data: DependencyData = {
            'modules': [],
            'functions_called': [],
            'global_vars': [],
            'classes_used': []
        }

        extractors = {
            'modules': ImportExtractor,
            'functions_called': FunctionCallExtractor,
            'global_vars': GlobalVarExtractor,
            'classes_used': ClassUsageExtractor
        }

        for key, extractor in extractors.items():
            for node in ast.walk(self.tree):
                data[key].extend(extractor.extract(node))

        return data


def main():
    paths = get_file_paths(PROJET_DIR, SUPPORTED_FILE_TYPES)
    
    for path in paths:
        with open(path, "r") as source:
            code = source.read()
        print(code)
        analyzer = CodeAnalyzer(code)
        result = analyzer.analyze()
        function_codes = analyzer.get_all_functions()
        print(function_codes)
        class_codes = analyzer.get_all_classes()
        print(class_codes)
        break
if __name__ == "__main__":
    main()

#!/usr/bin/env python
# -*- coding: utf-8 -*-
"""
提供配置，单例
"""
import os

import openai
import yaml
from metagpt.const import PROJECT_ROOT
from metagpt.logs import logger
from metagpt.tools import SearchEngineType, WebBrowserEngineType
from metagpt.utils.singleton import Singleton


class NotConfiguredException(Exception):
    """Exception raised for errors in the configuration.

    Attributes:
        message -- explanation of the error
    """

    def __init__(self, message="The required configuration is not set"):
        self.message = message
        super().__init__(self.message)


class Config(metaclass=Singleton):
    """
    常规使用方法：
    config = Config("config.yaml")
    secret_key = config.get_key("MY_SECRET_KEY")
    print("Secret key:", secret_key)
    """

    _instance = None
    key_yaml_file = PROJECT_ROOT / "config/key.yaml"
    default_yaml_file = PROJECT_ROOT / "config/config.yaml"

    def __init__(self, yaml_file=default_yaml_file):
        self._configs = {}
        self._init_with_config_files_and_env(self._configs, yaml_file)
        logger.info("Config loading done.")
        self.global_proxy = self._get("GLOBAL_PROXY")
        self.openai_api_key = self._get("OPENAI_API_KEY")
        if not self.openai_api_key or "YOUR_API_KEY" == self.openai_api_key:
            raise NotConfiguredException("Set OPENAI_API_KEY first")

        self.openai_api_base = self._get("OPENAI_API_BASE")
        if not self.openai_api_base or "YOUR_API_BASE" == self.openai_api_base:
            openai_proxy = self._get("OPENAI_PROXY") or self.global_proxy
            if openai_proxy:
                openai.proxy = openai_proxy
            else:
                logger.info("Set OPENAI_API_BASE in case of network issues")
        self.openai_api_type = self._get("OPENAI_API_TYPE")
        self.openai_api_version = self._get("OPENAI_API_VERSION")
        self.openai_api_rpm = self._get("RPM", 3)
        self.openai_api_model = self._get("OPENAI_API_MODEL", "gpt-4")
        self.max_tokens_rsp = self._get("MAX_TOKENS", 2048)
        self.deployment_id = self._get("DEPLOYMENT_ID")

        self.claude_api_key = self._get('Anthropic_API_KEY')
        self.serpapi_api_key = self._get("SERPAPI_API_KEY")
        self.serper_api_key = self._get("SERPER_API_KEY")
        self.google_api_key = self._get("GOOGLE_API_KEY")
        self.google_cse_id = self._get("GOOGLE_CSE_ID")
        self.search_engine = self._get("SEARCH_ENGINE", SearchEngineType.SERPAPI_GOOGLE)
 
        self.web_browser_engine = WebBrowserEngineType(self._get("WEB_BROWSER_ENGINE", "playwright"))
        self.playwright_browser_type = self._get("PLAYWRIGHT_BROWSER_TYPE", "chromium")
        self.selenium_browser_type = self._get("SELENIUM_BROWSER_TYPE", "chrome")
      
        self.long_term_memory = self._get('LONG_TERM_MEMORY', False)
        if self.long_term_memory:
            logger.warning("LONG_TERM_MEMORY is True")
        self.max_budget = self._get("MAX_BUDGET", 10.0)
        self.total_cost = 0.0
        self.puppeteer_config = self._get("PUPPETEER_CONFIG","")
        self.mmdc = self._get("MMDC","mmdc")
        self.update_costs = self._get("UPDATE_COSTS",True)
        self.calc_usage = self._get("CALC_USAGE",True)

        

    def _init_with_config_files_and_env(self, configs: dict, yaml_file):
        """从config/key.yaml / config/config.yaml / env三处按优先级递减加载"""
        configs.update(os.environ)

        for _yaml_file in :
            if not _yaml_file.exists():
                continue

            # 加载本地 YAML 文件
            with open(_yaml_file, "r", encoding="utf-8") as file:
                yaml_data = yaml.safe_load(file)
                if not yaml_data:
                    continue
                os.environ.update({k: v for k, v in yaml_data.items() if isinstance(v, str)})
                configs.update(yaml_data)

    def _get(self, *args, **kwargs):
        return self._configs.get(*args, **kwargs)

    def get(self, key, *args, **kwargs):


{
    '__init__': '    def __init__(self, message="The required configuration is not set"):\n        self.message = 
message\n        super().__init__(self.message)\n\n',
    '_init_with_config_files_and_env': '    def _init_with_config_files_and_env(self, configs: dict, yaml_file):\n 
"""从config/key.yaml / config/config.yaml / env三处按优先级递减加载"""\n        configs.update(os.environ)\n\n     
for _yaml_file in [yaml_file, self.key_yaml_file]:\n            if not _yaml_file.exists():\n                
continue\n\n            # 加载本地 YAML 文件',
    '_get': '    def _get(self, *args, **kwargs):\n        return self._configs.get(*args, **kwargs)\n',
    'get': '    def get(self, key, *args, **kwargs):\n        """从config/key.yaml / config/config.yaml / 
env三处找值，找不到报错"""\n        value = self._get(key, *args, **kwargs)\n        if value is None:\n           
raise ValueError(f"Key \'{key}\' not found in environment variables or in the YAML file")'
}

{}

In [53]:
import ast
import tokenize
from io import BytesIO
from typing import List, Dict, Union, Optional, Any, Type, Callable
from typing_extensions import TypedDict
from rich import print

from file import get_file_paths

PROJECT_DIR = "./MetaGPT/metagpt"
SUPPORTED_FILE_TYPES = [".py"]

class DependencyData(TypedDict):
    modules: List[str]
    functions_called: List[str]
    global_vars: List[str]
    classes_used: List[str]
    function_codes: Dict[str, str]
    class_codes: Dict[str, str]
    
def extract_from_node(extractor: Type, node: ast.AST) -> List[str]:
    return extractor.extract(node)

class ImportExtractor:
    @staticmethod
    def extract(node: ast.AST) -> List[str]:
        if isinstance(node, ast.Import):
            return [n.name for n in node.names]
        elif isinstance(node, ast.ImportFrom):
            return [f"{node.module}.{n.name}" for n in node.names]
        return []

class FunctionCallExtractor:
    @staticmethod
    def extract(node: ast.AST) -> List[str]:
        if isinstance(node, ast.Call):
            return [node.func.id if isinstance(node.func, ast.Name) else node.func.attr]
        return []

class GlobalVarExtractor:
    @staticmethod
    def extract(node: ast.AST) -> List[str]:
        return node.names if isinstance(node, ast.Global) else []

class ClassUsageExtractor:
    @staticmethod
    def extract(node: ast.AST) -> List[str]:
        return [node.name] if isinstance(node, ast.ClassDef) else []

def extract_code_segment(code: str, node: Union[ast.FunctionDef, ast.ClassDef]) -> Optional[str]:
    # Use the line numbers provided by the ast node
    start_line = node.lineno - 1  # AST line numbers are 1-based, lists are 0-based
    end_line = node.end_lineno if hasattr(node, 'end_lineno') else start_line
    return '\n'.join(code.splitlines()[start_line:end_line + 1])

def analyze_code(code: str) -> DependencyData:
    tree = ast.parse(code)
    
    data: DependencyData = {
        'modules': [],
        'functions_called': [],
        'global_vars': [],
        'classes_used': [],
        'function_codes': {},
        'class_codes': {}
    }

    extractors = {
        'modules': ImportExtractor,
        'functions_called': FunctionCallExtractor,
        'global_vars': GlobalVarExtractor,
        'classes_used': ClassUsageExtractor
    }

    for key, extractor in extractors.items():
        for node in ast.walk(tree):
            data[key].extend(extract_from_node(extractor, node))

    for node in tree.body:  # Only top-level nodes
        if isinstance(node, ast.FunctionDef) and not node.name.startswith('_'):
            data['function_codes'][node.name] = extract_code_segment(code, node)
        elif isinstance(node, ast.ClassDef):
            data['class_codes'][node.name] = extract_code_segment(code, node)

    return data



def main():
    paths = get_file_paths(PROJECT_DIR, SUPPORTED_FILE_TYPES)
    
    for path in paths:
        with open(path, "r") as source:
            code = source.read()
        
        result = analyze_code(code)
        
        print(result['function_codes'])
        print(result['class_codes'])

if __name__ == "__main__":
    main()

{}

{
    'NotConfiguredException': 'class NotConfiguredException(Exception):\n    """Exception raised for errors in the 
configuration.\n\n    Attributes:\n        message -- explanation of the error\n    """\n\n    def __init__(self, 
message="The required configuration is not set"):\n        self.message = message\n        
super().__init__(self.message)\n',
    'Config': 'class Config(metaclass=Singleton):\n    """\n    常规使用方法：\n    config = 
Config("config.yaml")\n    secret_key = config.get_key("MY_SECRET_KEY")\n    print("Secret key:", secret_key)\n    
"""\n\n    _instance = None\n    key_yaml_file = PROJECT_ROOT / "config/key.yaml"\n    default_yaml_file = 
PROJECT_ROOT / "config/config.yaml"\n\n    def __init__(self, yaml_file=default_yaml_file):\n        self._configs 
= {}\n        self._init_with_config_files_and_env(self._configs, yaml_file)\n        logger.info("Config loading 
done.")\n        self.global_proxy = self._get("GLOBAL_PROXY")\n        self.openai_api_key = 
self._get("OPENAI_API_KEY")\n        if not self.openai_api_key or "YOUR_API_KEY" == self.openai_api_key:\n        
raise NotConfiguredException("Set OPENAI_API_KEY first")\n\n        self.openai_api_base = 
self._get("OPENAI_API_BASE")\n        if not self.openai_api_base or "YOUR_API_BASE" == self.openai_api_base:\n    
openai_proxy = self._get("OPENAI_PROXY") or self.global_proxy\n            if openai_proxy:\n                
openai.proxy = openai_proxy\n            else:\n                logger.info("Set OPENAI_API_BASE in case of network
issues")\n        self.openai_api_type = self._get("OPENAI_API_TYPE")\n        self.openai_api_version = 
self._get("OPENAI_API_VERSION")\n        self.openai_api_rpm = self._get("RPM", 3)\n        self.openai_api_model =
self._get("OPENAI_API_MODEL", "gpt-4")\n        self.max_tokens_rsp = self._get("MAX_TOKENS", 2048)\n        
self.deployment_id = self._get("DEPLOYMENT_ID")\n\n        self.claude_api_key = self._get(\'Anthropic_API_KEY\')\n
self.serpapi_api_key = self._get("SERPAPI_API_KEY")\n        self.serper_api_key = self._get("SERPER_API_KEY")\n   
self.google_api_key = self._get("GOOGLE_API_KEY")\n        self.google_cse_id = self._get("GOOGLE_CSE_ID")\n       
self.search_engine = self._get("SEARCH_ENGINE", SearchEngineType.SERPAPI_GOOGLE)\n \n        
self.web_browser_engine = WebBrowserEngineType(self._get("WEB_BROWSER_ENGINE", "playwright"))\n        
self.playwright_browser_type = self._get("PLAYWRIGHT_BROWSER_TYPE", "chromium")\n        self.selenium_browser_type
= self._get("SELENIUM_BROWSER_TYPE", "chrome")\n      \n        self.long_term_memory = 
self._get(\'LONG_TERM_MEMORY\', False)\n        if self.long_term_memory:\n            
logger.warning("LONG_TERM_MEMORY is True")\n        self.max_budget = self._get("MAX_BUDGET", 10.0)\n        
self.total_cost = 0.0\n        self.puppeteer_config = self._get("PUPPETEER_CONFIG","")\n        self.mmdc = 
self._get("MMDC","mmdc")\n        self.update_costs = self._get("UPDATE_COSTS",True)\n        self.calc_usage = 
self._get("CALC_USAGE",True)\n\n        \n\n    def _init_with_config_files_and_env(self, configs: dict, 
yaml_file):\n        """从config/key.yaml / config/config.yaml / env三处按优先级递减加载"""\n        
configs.update(os.environ)\n\n        for _yaml_file in [yaml_file, self.key_yaml_file]:\n            if not 
_yaml_file.exists():\n                continue\n\n            # 加载本地 YAML 文件\n            with 
open(_yaml_file, "r", encoding="utf-8") as file:\n                yaml_data = yaml.safe_load(file)\n               
if not yaml_data:\n                    continue\n                os.environ.update({k: v for k, v in 
yaml_data.items() if isinstance(v, str)})\n                configs.update(yaml_data)\n\n    def _get(self, *args, 
**kwargs):\n        return self._configs.get(*args, **kwargs)\n\n    def get(self, key, *args, **kwargs):\n        
"""从config/key.yaml / config/config.yaml / env三处找值，找不到报错"""\n        value = self._get(key, *args

{}

{
    'SoftwareCompany': 'class SoftwareCompany(BaseModel):\n    """\n    Software Company: Possesses a team, SOP 
(Standard Operating Procedures), and a platform for instant messaging,\n    dedicated to writing executable code.\n
"""\n    environment: Environment = Field(default_factory=Environment)\n    investment: float = 
Field(default=10.0)\n    idea: str = Field(default="")\n\n    class Config:\n        arbitrary_types_allowed = 
True\n\n    def hire(self, roles: list[Role]):\n        """Hire roles to cooperate"""\n        
self.environment.add_roles(roles)\n\n    def invest(self, investment: float):\n        """Invest company. raise 
NoMoneyException when exceed max_budget."""\n        self.investment = investment\n        CONFIG.max_budget = 
investment\n        logger.info(f\'Investment: ${investment}.\')\n\n    def _check_balance(self):\n        if 
CONFIG.total_cost > CONFIG.max_budget:\n            raise NoMoneyException(CONFIG.total_cost, f\'Insufficient 
funds: {CONFIG.max_budget}\')\n\n    def start_project(self, idea):\n        """Start a project from publishing 
boss requirement."""\n        self.idea = idea\n        self.environment.publish_message(Message(role="BOSS", 
content=idea, cause_by=BossRequirement))\n\n    def _save(self):\n        logger.info(self.json())\n\n    async def
run(self, n_round=3):\n        """Run company until target round or no money"""\n        while n_round > 0:\n      
# self._save()\n            n_round -= 1\n            logger.debug(f"{n_round=}")\n            
self._check_balance()\n            await self.environment.run()\n        return self.environment.history'
}

{
    'print_classes_and_functions': 'def print_classes_and_functions(module):\n    """FIXME: NOT WORK.. """\n    for
name, obj in inspect.getmembers(module):\n        if inspect.isclass(obj):\n            print(f\'Class: {name}\')\n
elif inspect.isfunction(obj):\n            print(f\'Function: {name}\')\n        else:\n            print(name)\n\n
print(dir(module))\n'
}

...

{}

{
    'UTGenerator': 'class UTGenerator:\n    """UT生成器：通过API文档构造UT"""\n\n    def __init__(self, 
swagger_file: str, ut_py_path: str, questions_path: str,\n                 chatgpt_method: str = "API", 
template_prefix=YFT_PROMPT_PREFIX) -> None:\n        """初始化UT生成器\n\n        Args:\n            swagger_file: 
swagger路径\n            ut_py_path: 用例存放路径\n            questions_path: 模版存放路径，便于后续排查\n        
chatgpt_method: API\n            template_prefix: 使用模版，默认使用YFT_UT_PROMPT\n        """\n        
self.swagger_file = swagger_file\n        self.ut_py_path = ut_py_path\n        self.questions_path = 
questions_path\n        assert chatgpt_method in ["API"], "非法chatgpt_method"\n        self.chatgpt_method = 
chatgpt_method\n\n        # ICL: In-Context Learning，这里给出例子，要求GPT模仿例子\n        self.icl_sample = 
ICL_SAMPLE\n        self.template_prefix = template_prefix\n\n    def get_swagger_json(self) -> dict:\n        
"""从本地文件加载Swagger JSON"""\n        with open(self.swagger_file, "r", encoding="utf-8") as file:\n           
swagger_json = json.load(file)\n        return swagger_json\n\n    def __para_to_str(self, prop, required, 
name=""):\n        name = name or prop["name"]\n        ptype = prop["type"]\n        title = prop.get("title", 
"")\n        desc = prop.get("description", "")\n        return f\'{name}\\t{ptype}\\t{"是" if required else 
"否"}\\t{title}\\t{desc}\'\n\n    def _para_to_str(self, prop):\n        required = prop.get("required", False)\n  
return self.__para_to_str(prop, required)\n\n    def para_to_str(self, name, prop, prop_object_required):\n        
required = name in prop_object_required\n        return self.__para_to_str(prop, required, name)\n\n    def 
build_object_properties(self, node, prop_object_required, level: int = 0) -> str:\n        
"""递归输出object和array[object]类型的子属性\n\n        Args:\n            node (_type_): 子项的值\n            
prop_object_required (_type_): 是否必填项\n            level: 当前递归深度\n        """\n\n        doc = ""\n\n    
def dive_into_object(node):\n            """如果是object类型，递归输出子属性"""\n            if node.get("type") ==
"object":\n                sub_properties = node.get("properties", {})\n                return 
self.build_object_properties(sub_properties, prop_object_required, level=level + 1)\n            return ""\n\n     
if node.get("in", "") in ["query", "header", "formData"]:\n            doc += f\'{"\t" * 
level}{self._para_to_str(node)}\\n\'\n            doc += dive_into_object(node)\n            return doc\n\n        
for name, prop in node.items():\n            doc += f\'{"\t" * level}{self.para_to_str(name, prop, 
prop_object_required)}\\n\'\n            doc += dive_into_object(prop)\n            if prop["type"] == "array":\n  
items = prop.get("items", {})\n                doc += dive_into_object(items)\n        return doc\n\n    def 
get_tags_mapping(self) -> dict:\n        """处理tag与path\n\n        Returns:\n            Dict: tag: 
path对应关系\n        """\n        swagger_data = self.get_swagger_json()\n        paths = swagger_data["paths"]\n 
tags = {}\n\n        for path, path_obj in paths.items():\n            for method, method_obj in 
path_obj.items():\n                for tag in method_obj["tags"]:\n                    if tag not in tags:\n       
tags[tag] = {}\n                    if path not in tags[tag]:\n                        tags[tag][path] = {}\n      
tags[tag][path][method] = method_obj\n\n        return tags\n\n    def generate_ut(self, include_tags) -> bool:\n  
"""生成用例文件"""\n        tags = self.get_tags_mapping()\n        for tag, paths in tags.items():\n            if
include_tags is None or tag in include_tags:\n                self._generate_ut(tag, paths)\n        return 
True\n\n    def build_api_doc(self, node: dict, path: str, method: str) -> str:\n        summary = 
node["summary"]\n\n        doc = f"接口名称：{summary}\\n接口路径：{path}\\nMethod：{method.upper()}\\n"\n        
doc += "\\n请求参数：\\n"\n        if "parameters" in node:\n            param

{}

{
    'Translator': "class Translator:\n\n    @classmethod\n    def translate_prompt(cls, original, lang='中文'):\n  
return prompt.format(LANG=lang, ORIGINAL=original)"
}

AttributeError: 'Subscript' object has no attribute 'attr'